In [1]:
import numpy as np
from scipy.stats import bernoulli
import matplotlib as plt
from matplotlib import pyplot
from scipy.interpolate import make_interp_spline
import random 
import math

In [3]:
class BernoulliBandit :
    def __init__ (self , means):
        for i in means:
            assert(i <= 1 and i >= 0)
        self.means = means
        self.k = len(means)
        self.best_mean = max(means)
        self.regret = 0
# Accepts a parameter 0 <= a <= K -1 and returns the
# realisation of random variable X with P(X = 1) being
# the mean of the (a +1) th arm .
    def pull (self , a):
        self.regret += (self.best_mean - self.means[a])
        return bernoulli.rvs(self.means[a], size=1)

In [25]:
class GaussianBandit:
    def __init__ (self , means):
        self.means = means
        self.k = len(means)
        self.best_mean = max(means)
        self.regret = 0
        
    def pull (self , a):
        self.regret += (self.best_mean - self.means[a])
        return random.gauss(self.means[a], 1)


In [12]:
def UCB(delta, bandit, n):
    k = bandit.k #number of arms
    upper_bounds = float('inf') * np.ones(k) #set upper bounds to infinity
    result = [] #to store pairs of a_t, x_t
    means = np.zeros((k,2)) #to store means and T_i
    for i in range(n):
        arm = np.argmax(upper_bounds)
        reward = bandit.pull(arm)
        result.append((arm, reward))
        means[arm,0] = (means[arm,0] * means[arm,1] + reward)/(means[arm,1]+1)
        means[arm,1] += 1
        upper_bounds[arm] = means[arm,0] + np.sqrt(2*np.log(1/delta)/means[arm,1])
    return result, bandit.regret

## Q1
### a)
$\mathbb{P}\Bigl(\hat{\mu}-\mu \geq \sqrt{\frac{2log(1/\delta)}{T}}\Bigr) = \sum_{n=1}^\infty \mathbb{E}\Bigl[\{\mathbb{1}\{T=n\}\mathbb{1}\Bigl\{\sum_{t=1}^n(X_t-\mu)\geq\sqrt{2nlog(1/\delta)}\Bigr\}\Bigr]
\\ =\sum_{n=1}^\infty \mathbb{E}\Bigl[\mathbb{E}\bigl[\mathbb{1}\{T=n\}\mathbb{1}\Bigl\{\sum_{t=1}^n(X_t-\mu)\geq \sqrt{2nlog(1/\delta)}\Bigr\}|T\bigr]\Bigr]
\\ =\sum_{n=1}^\infty \mathbb{E}\Bigl[\mathbb{1}\{T=n\}\mathbb{E}\bigl[\mathbb{1}\Bigl\{\sum_{t=1}^T(X_t-\mu)\geq \sqrt{2Tlog(1/\delta)}\Bigr\}|T\bigr]\Bigr]
\\ \leq \sum_{n=1}^\infty\mathbb{E}[\mathbb{1}\{T=n\}\delta]
\\ = \delta$

### b) (discuss in the meeting)
Not sure how to use law of iterated logarithm;

$T= min\bigl\{n:\sum_{t=1}^n(X_t-\mu)\geq\sqrt{2nlog(1/\delta)}\bigr\}$

$T<\infty$ because $\sum_{t=1}^n(X_t-\mu)$ grows just as fast as $\sqrt{2nloglogn}$ which grows faster than $\sqrt{2nlog(1/\delta)}$  but how to express that mathematically?

### c) 
$\mathbb{P}\Bigl(\hat{\mu}-\mu \geq \sqrt{\frac{2log(T(T+1)/\delta)}{T}}\Bigr) \leq \mathbb{P}\Bigl(\exists n : \sum_{t=1}^n(X_t-\mu)\geq \sqrt{2nlog(n(n+1)/\delta)}\Bigr)
\\ \leq \sum_{n=1}^\infty \frac{\delta}{n(n+1)}
\\ = \delta$
